# Agentic RAG Playground by LlamaIndex

In [1]:
import datasets
from llama_index.core.schema import Document

In [2]:
# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        text="\n".join([
            f"Name: {guest_dataset['name'][i]}",
            f"Relation: {guest_dataset['relation'][i]}",
            f"Description: {guest_dataset['description'][i]}",
            f"Email: {guest_dataset['email'][i]}"
        ]),
        metadata={"name": guest_dataset['name'][i]}
    )
    for i in range(len(guest_dataset))
]

In [3]:
# Initialize the tool
from src.tools import GuestInfoRetrieverTool
guest_info_tool = GuestInfoRetrieverTool(docs)

In [4]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.ollama import Ollama
from llama_index.core.tools import FunctionTool, ToolOutput
from src.tools import GuestInfoRetrieverTool

# Set up the ReAct agent with a custom prompt to work around tool calling limitations
from llama_index.core.prompts.default_prompts import DEFAULT_REACT_PROMPT

# Initialize the guest info retriever tool
guest_info_retriever = GuestInfoRetrieverTool(docs)

# Create a proper FunctionTool from the method
guest_info_tool = FunctionTool.from_defaults(
    fn=guest_info_retriever.get_guest_info,
    name="guest_info",
    description="Retrieve information about guests attending the gala by their name or details"
)

# Initialize the LLM which one is available in the local system
llm = Ollama(model="llama3:latest")

# Create ReAct agent
alfred = ReActAgent.from_tools(
    [guest_info_tool],
    llm=llm,
    verbose=True,
    system_prompt=(
        "You are Alfred, an attentive and knowledgeable butler at a prestigious gala. "
        "Your job is to provide information about the guests when asked. "
        "Use the guest_info tool to retrieve information about guests when needed."
    ),
)

# Example query for the gala agent
response = await alfred.aquery("Tell me about our guest named 'Lady Ada Lovelace'.")

print("🎩 Alfred's Response:")
print(response.response)

ImportError: cannot import name 'DEFAULT_REACT_PROMPT' from 'llama_index.core.prompts.default_prompts' (/Users/gamzekecibas/opt/anaconda3/envs/agentenv/lib/python3.9/site-packages/llama_index/core/prompts/default_prompts.py)